In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
sst_col_list = [c for c in add_train.columns if 'sst' in c]
sst_col_list

['sst-2010-1',
 'sst-2010-2',
 'sst-2010-3',
 'sst-2010-4',
 'sst-2010-5',
 'sst-2010-6',
 'sst-2010-7',
 'sst-2010-8',
 'sst-2010-9',
 'sst-2010-10']

In [4]:
climate_type_dict = {'1':'BSk',
 '9': 'Dfb',
 '4': 'Cfa',
 '7': 'Csb',
 '8': 'Dfa',
 '3': 'BWk',
 '10': 'Dfc',
 '2': 'BWh',
 '6': 'Csa',
 '11': 'Dsb',
 '0': 'BSh',
 '5': 'Cfb',
 '12': 'Dsc',
 '13': 'Dwa',
 '14': 'Dwb'}

In [5]:
all_cols = add_train.columns.tolist()

In [6]:
target=["contest-tmp2m-14d__tmp2m"]

In [7]:
train_df =  pd.read_csv('fe_v1_train.csv')
test_df  =  pd.read_csv('fe_v1_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [8]:
exclude_cols = ['index', 'startdate']
best_cols = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(best_cols))

Main features: 63


In [9]:
train_df[sst_col_list] = add_train[sst_col_list]
test_df[sst_col_list] = add_test[sst_col_list]

In [10]:
for c in all_cols:
    if 'contest-precip-14d' in c:
        print(c)

contest-precip-14d__precip


In [11]:
def calculate_season(month):
    if month == 12 or month == 1 or month == 2:
        return 4
    if month == 9 or month == 10 or month == 11:
        return 3
    if month == 7 or month == 8 or month == 6:
        return 2
    if month == 3 or month == 4 or month == 5:
        return 1

In [12]:
train_df['month'] = add_train['month']
test_df['month'] = add_test['month']

train_df['season'] = train_df['month'].apply(calculate_season)
test_df['season'] = test_df['month'].apply(calculate_season)

train_df['loc_group'] = add_train['loc_group']
test_df['loc_group'] = add_test['loc_group']

train_df['year'] = add_train['year']
test_df['year'] = add_test['year']

def map_climate_type(x):
    return climate_type_dict[str(x)]

train_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)
test_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)

precip_col = 'contest-precip-14d__precip'
wind_col = 'contest-wind-h500-14d__wind-hgt-500'

group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']


In [13]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
stt_col = 'sst-2010-1'
sst_col_5 = 'sst-2010-5'
sst_col_3 = 'sst-2010-3'
sst_col_4 = 'sst-2010-4'

In [14]:
print(train_df[pres_col].min(), train_df[pres_col].max())
print(test_df[pres_col].min(), test_df[pres_col].max())

74839.94 102845.57
75925.02 102119.93


In [15]:
print(train_df[stt_col].min(), train_df[stt_col].max())
print(test_df[stt_col].min(), test_df[stt_col].max())

-219.87 352.2
89.73 345.33


In [16]:
print(train_df[sst_col_5].min(), train_df[sst_col_5].max())
print(test_df[sst_col_5].min(), test_df[sst_col_5].max())

-32.81 40.97
-4.62 21.5


In [17]:
print(train_df[precip_col].min(), train_df[precip_col].max())
print(test_df[precip_col].min(), test_df[precip_col].max())

0.0 502.63
0.0 439.54


In [18]:
# train_df[pres_col] = train_df[pres_col] - 100000
# test_df[pres_col] = test_df[pres_col] - 100000

In [19]:
print(train_df[slp_col].min(), train_df[slp_col].max())
print(test_df[slp_col].min(), test_df[slp_col].max())

100379.21 103275.84
100593.22 102258.27


In [20]:
print(train_df[wind_col].min(), train_df[wind_col].max())
print(test_df[wind_col].min(), test_df[wind_col].max())

5289.25 5958.2
5504.59 5925.56


In [21]:
train_df[wind_col] = train_df[wind_col] - 5000
test_df[wind_col] = test_df[wind_col] - 5000

In [22]:
train_df[slp_col] = train_df[slp_col] - 100000
test_df[slp_col] = test_df[slp_col] - 100000

In [23]:
check_cols = [c for c in train_df.columns if c not in target and c not in exclude_cols]

df = pd.concat([train_df[check_cols], test_df[check_cols]], axis=0)
df.shape

(407088, 74)

In [24]:
def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

In [25]:
def assign_train_test_df(my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()

In [26]:
first_slp_df = create_df_by_group_and_agg(df, season_group_cols, "first", slp_col, "first_slp")
first_slp_df.head(3)

Nan


,loc_group,climate_type,year,season,first_slp
0,0,BSh,2014,3,1352.08
1,0,BSh,2014,4,2072.81
2,0,BSh,2015,1,1917.24


In [27]:
first_wind_df = create_df_by_group_and_agg(df, season_group_cols, "first", wind_col, "first_wind")
first_wind_df.head(3)

Nan


,loc_group,climate_type,year,season,first_wind
0,0,BSh,2014,3,899.66
1,0,BSh,2014,4,847.35
2,0,BSh,2015,1,804.47


In [28]:
first_precip_df = create_df_by_group_and_agg(df, season_group_cols, "first", precip_col, "first_precip")
first_precip_df.head(3)

Nan


,loc_group,climate_type,year,season,first_precip
0,0,BSh,2014,3,94.31
1,0,BSh,2014,4,25.53
2,0,BSh,2015,1,31.94


In [29]:
first_ccsm3_df = create_df_by_group_and_agg(df, season_group_cols, "first", ccsm3_col, "first_ccsm3")
first_ccsm3_df.head(3)

Nan


,loc_group,climate_type,year,season,first_ccsm3
0,0,BSh,2014,3,27.57
1,0,BSh,2014,4,6.66
2,0,BSh,2015,1,15.25


In [30]:
first_pres_df = create_df_by_group_and_agg(df, season_group_cols, "first", ccsm3_col, "first_pres")
first_pres_df.head(3)

Nan


,loc_group,climate_type,year,season,first_pres
0,0,BSh,2014,3,27.57
1,0,BSh,2014,4,6.66
2,0,BSh,2015,1,15.25


In [31]:
def assign_train_test_sst_df(df, my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()
    return df


def create_sst_feature(df, sst_col_name, suffix_num, season_group_cols):
    first_df = create_df_by_group_and_agg(df, season_group_cols, "first", sst_col_name, f"first_sst_{suffix_num}")
    min_df = create_df_by_group_and_agg(df, season_group_cols, "min", sst_col_name, f"min_sst_{suffix_num}")
    max_df = create_df_by_group_and_agg(df, season_group_cols, "max", sst_col_name, f"max_sst_{suffix_num}")
    
    df = df.merge(first_df, how='left', on=season_group_cols)
    df[f'diff_sst_{suffix_num}_first'] = df[sst_col_name] - df[f"first_sst_{suffix_num}"]
    print("Bug: ", df[f'diff_sst_{suffix_num}_first'].isnull().sum())
    df = assign_train_test_sst_df(df, f'diff_sst_{suffix_num}_first')
    
    df = df.merge(min_df, how='left', on=season_group_cols)
    df[f'diff_sst_{suffix_num}_min'] = df[sst_col_name] - df[f"min_sst_{suffix_num}"]
    df = assign_train_test_sst_df(df, f'diff_sst_{suffix_num}_min')
    
    df = df.merge(max_df, how='left', on=season_group_cols)
    df[f'diff_sst_{suffix_num}_max'] = df[sst_col_name] - df[f"max_sst_{suffix_num}"]
    df = assign_train_test_sst_df(df, f'diff_sst_{suffix_num}_max')
    
    df[f'range_sst_{suffix_num}'] = df[f"max_sst_{suffix_num}"] - df[f"min_sst_{suffix_num}"]
    df = assign_train_test_sst_df(df, f'range_sst_{suffix_num}')
    
    df[f'scale_sst_{suffix_num}'] = (df[sst_col_name] - df[f'diff_sst_{suffix_num}_min']) / df[f'range_sst_{suffix_num}']
    df = assign_train_test_sst_df(df, f'scale_sst_{suffix_num}')
    
    return [df, f'diff_sst_{suffix_num}_first',
    f'diff_sst_{suffix_num}_min',
    f'diff_sst_{suffix_num}_max',
    f'range_sst_{suffix_num}', f'scale_sst_{suffix_num}']
    


In [32]:
# first_sst_df = create_df_by_group_and_agg(df, season_group_cols, "first", stt_col, "first_sst")
# first_sst_df.head(3)

In [33]:
# first_sst_5_df = create_df_by_group_and_agg(df, season_group_cols, "first", sst_col_5, "first_sst_5")
# first_sst_5_df.head(3)

In [34]:
# first_sst_3_df = create_df_by_group_and_agg(df, season_group_cols, "first", sst_col_3, "first_sst_3")
# first_sst_3_df.head(3)

In [35]:
# first_sst_4_df = create_df_by_group_and_agg(df, season_group_cols, "first", sst_col_4, "first_sst_4")
# first_sst_4_df.head(3)

In [36]:
# min_sst_df = create_df_by_group_and_agg(df, season_group_cols, "min", stt_col, "min_sst")
# min_sst_df.head(3)

In [37]:
# min_sst_5_df = create_df_by_group_and_agg(df, season_group_cols, "min", sst_col_5, "min_sst_5")
# min_sst_5_df.head(3)

In [38]:
# min_sst_3_df = create_df_by_group_and_agg(df, season_group_cols, "min", sst_col_3, "min_sst_3")
# min_sst_3_df.head(3)

In [39]:
# min_sst_4_df = create_df_by_group_and_agg(df, season_group_cols, "min", sst_col_4, "min_sst_4")
# min_sst_4_df.head(3)

In [40]:
# max_sst_df = create_df_by_group_and_agg(df, season_group_cols, "max", stt_col, "max_sst")
# max_sst_df.head(3)

In [41]:
# max_sst_5_df = create_df_by_group_and_agg(df, season_group_cols, "max", sst_col_5, "max_sst_5")
# max_sst_5_df.head(3)

In [42]:
# max_sst_3_df = create_df_by_group_and_agg(df, season_group_cols, "max", sst_col_3, "max_sst_3")
# max_sst_3_df.head(3)

In [43]:
# max_sst_4_df = create_df_by_group_and_agg(df, season_group_cols, "max", sst_col_4, "max_sst_4")
# max_sst_4_df.head(3)

In [44]:
first_slp_df_month = create_df_by_group_and_agg(df, group_cols, "first", slp_col, "first_slp_month")
first_slp_df_month.head(3)

Nan


,loc_group,climate_type,year,season,month,first_slp_month
0,0,BSh,2014,3,9,1352.08
1,0,BSh,2014,3,10,1254.96
2,0,BSh,2014,3,11,1952.19


In [45]:
df = df.merge(first_slp_df, how='left', on=season_group_cols)
df['diff_slp_first'] = df[slp_col] - df['first_slp']
assign_train_test_df('diff_slp_first')

In [46]:
df = df.merge(first_precip_df, how='left', on=season_group_cols)
df['diff_precip_first'] = df[precip_col] - df['first_precip']
assign_train_test_df('diff_precip_first')

In [47]:
# df = df.merge(first_sst_df, how='left', on=season_group_cols)
# df['diff_sst_first'] = df[stt_col] - df['first_sst']
# assign_train_test_df('diff_sst_first')

In [48]:
# df = df.merge(first_sst_5_df, how='left', on=season_group_cols)
# df['diff_sst_5_first'] = df[sst_col_5] - df['first_sst_5']
# assign_train_test_df('diff_sst_5_first')

In [49]:
# df = df.merge(first_sst_3_df, how='left', on=season_group_cols)
# df['diff_sst_3_first'] = df[sst_col_3] - df['first_sst_3']
# assign_train_test_df('diff_sst_3_first')

In [50]:
# df = df.merge(first_sst_4_df, how='left', on=season_group_cols)
# df['diff_sst_4_first'] = df[sst_col_4] - df['first_sst_4']
# assign_train_test_df('diff_sst_4_first')

In [51]:
# df = df.merge(min_sst_df, how='left', on=season_group_cols)
# df['diff_sst_min'] = df[stt_col] - df['min_sst']
# assign_train_test_df('diff_sst_min')

In [52]:
# df = df.merge(max_sst_df, how='left', on=season_group_cols)
# df['diff_sst_max'] = df['max_sst'] - df[stt_col]
# assign_train_test_df('diff_sst_max')

In [53]:
# df = df.merge(max_sst_5_df, how='left', on=season_group_cols)
# df['diff_sst_5_max'] = df[sst_col_5] - df['max_sst_5']
# assign_train_test_df('diff_sst_5_max')

In [54]:
# df = df.merge(max_sst_3_df, how='left', on=season_group_cols)
# df['diff_sst_3_max'] = df[sst_col_3] - df['max_sst_3']
# assign_train_test_df('diff_sst_3_max')

In [55]:
# df = df.merge(max_sst_4_df, how='left', on=season_group_cols)
# df['diff_sst_4_max'] = df[sst_col_4] - df['max_sst_4']
# assign_train_test_df('diff_sst_4_max')

In [56]:
# df = df.merge(min_sst_5_df, how='left', on=season_group_cols)
# df['diff_sst_5_min'] = df[sst_col_5] - df['min_sst_5']
# assign_train_test_df('diff_sst_5_min')

In [57]:
# df = df.merge(min_sst_3_df, how='left', on=season_group_cols)
# df['diff_sst_3_min'] = df[sst_col_3] - df['min_sst_3']
# assign_train_test_df('diff_sst_3_min')

In [58]:
# df = df.merge(min_sst_4_df, how='left', on=season_group_cols)
# df['diff_sst_4_min'] = df[sst_col_3] - df['min_sst_4']
# assign_train_test_df('diff_sst_4_min')

In [59]:
df = df.merge(first_pres_df, how='left', on=season_group_cols)
df['diff_pres_first'] = df[pres_col] - df['first_pres']
assign_train_test_df('diff_pres_first')

In [60]:
df = df.merge(first_wind_df, how='left', on=season_group_cols)
df['diff_wind_first'] = df[wind_col] - df['first_wind']
assign_train_test_df('diff_wind_first')

In [61]:
df = df.merge(first_ccsm3_df, how='left', on=season_group_cols)
df['diff_ccsm3_first'] = df[ccsm3_col] - df['first_ccsm3']
assign_train_test_df('diff_ccsm3_first')

In [62]:
# df[f'{precip_col}_bins'] = pd.qcut(df[precip_col], q=5, labels=[f'{precip_col}_{i}' for i in range(5)])
# le = LabelEncoder().fit(np.unique(df[f'{precip_col}_bins'].unique().tolist()))
# df[f'{precip_col}_bins'] = le.transform(df[f'{precip_col}_bins']) + 1
# df[f'{precip_col}_bins'] = df[f'{precip_col}_bins'].replace(np.nan, 0).astype('int').astype('category')
# assign_train_test_df(f'{precip_col}_bins')

In [63]:
# df['range_sst'] = df['max_sst'] - df['min_sst']
# assign_train_test_df('range_sst')

In [64]:
# df['range_sst_5'] = df['max_sst_5'] - df['min_sst_5']
# assign_train_test_df('range_sst_5')

In [65]:
# df['range_sst_3'] = df['max_sst_3'] - df['min_sst_3']
# assign_train_test_df('range_sst_3')

In [66]:
# df['range_sst_4'] = df['max_sst_4'] - df['min_sst_4']
# assign_train_test_df('range_sst_4')

In [67]:
sst_col_list

['sst-2010-1',
 'sst-2010-2',
 'sst-2010-3',
 'sst-2010-4',
 'sst-2010-5',
 'sst-2010-6',
 'sst-2010-7',
 'sst-2010-8',
 'sst-2010-9',
 'sst-2010-10']

In [68]:
sst_10_res = create_sst_feature(df, 'sst-2010-10', 10, season_group_cols)
df = sst_10_res[0]
sst_feature_10 = sst_10_res[1:]

sst_9_res = create_sst_feature(df, 'sst-2010-9', 9, season_group_cols)
df = sst_9_res[0]
sst_feature_9 = sst_9_res[1:]

sst_8_res = create_sst_feature(df, 'sst-2010-8', 8, season_group_cols)
df = sst_8_res[0]
sst_feature_8 = sst_8_res[1:]

sst_7_res = create_sst_feature(df, 'sst-2010-7', 7, season_group_cols)
df = sst_7_res[0]
sst_feature_7 = sst_7_res[1:]

sst_6_res = create_sst_feature(df, 'sst-2010-6', 6, season_group_cols)
df = sst_6_res[0]
sst_feature_6 = sst_6_res[1:]

sst_1_res = create_sst_feature(df, 'sst-2010-1', 1, season_group_cols)
df = sst_1_res[0]
sst_feature_1 = sst_1_res[1:]

sst_3_res = create_sst_feature(df, 'sst-2010-3', 3, season_group_cols)
df = sst_3_res[0]
sst_feature_3 = sst_3_res[1:]

sst_2_res = create_sst_feature(df, 'sst-2010-2', 2, season_group_cols)
df = sst_2_res[0]
sst_feature_2 = sst_2_res[1:]


sst_4_res = create_sst_feature(df, 'sst-2010-4', 4, season_group_cols)
df = sst_4_res[0]
sst_feature_4 = sst_4_res[1:]


sst_5_res = create_sst_feature(df, 'sst-2010-5', 5, season_group_cols)
df = sst_5_res[0]
sst_feature_5 = sst_5_res[1:]


Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0
Nan
Nan
Nan
Bug:  0


In [69]:
features = best_cols + [
    'diff_slp_first',
    'diff_wind_first',
#     'diff_sst_first',
    'diff_precip_first',
#     'diff_sst_min',
#     'diff_sst_max',
#     'range_sst',
#     'diff_sst_5_first',
#     'diff_sst_5_min',
#     'diff_sst_5_max',
#     'range_sst_5',
#     'diff_sst_3_first',
#     'diff_sst_3_min',
#     'diff_sst_3_max',
#     'range_sst_3',
#     'diff_sst_4_first',
#     'diff_sst_4_min',
#     'diff_sst_4_max',
#     'range_sst_4'
] + sst_feature_10 + sst_feature_9 + sst_feature_8 + sst_feature_7 + sst_feature_6 + sst_feature_1 + sst_feature_2 + sst_feature_3 + sst_feature_4 + sst_feature_5

In [70]:
features

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [71]:
test_df['index'] = add_test['index']

In [73]:
'climate_type' in train_df.columns

True

In [74]:
train_df.to_csv('./fe_v2_train_076.csv', index=False)
test_df.to_csv('./fe_v2_test_076.csv', index=False)

In [ ]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v4.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)